In [ ]:
pkgs <- c("Seurat","Signac","ArchR","tidyverse","patchwork",
          "ggalluvial","clusterProfiler","org.Mm.eg.db","TxDb.Mmusculus.UCSC.mm10.knownGene",
          "parallel",
          "BSgenome.Mmusculus.UCSC.mm10",
          "TFBSTools",
          "chromVARmotifs",
          "motifmatchr",
          "magrittr","viridis","igraph", "clustree", "reticulate", "Rsamtools")
suppressWarnings(suppressPackageStartupMessages(lapply(pkgs,library,character.only=TRUE)))
# source functions
source_python("atac/Python/utils.py")
source("atac/R/function.R")

# This is the sampleId that you will have to give to the Eykthyr object
sampleId <- "spatialATACRNA"
color_list<-c("1"="#644498","2"="#E94F2F","3"="#488CAD","4"="#D6ACCF","5"="#207639",
              "6"="#EF7D18","7"="#7184C1","8"="#70B1D7","9"="#DCBE70","10"="#A66C22",
              "11"="#1A7F85","12"="#ED7C7A","13"="#A8CD92","14"="#A91D30","15"="#F1CC32",
              "16"="#E6E754","17"="#063D20","18"="#8dd3c8","19"="#b31631","20"="#fbd326"
              )

set.seed(123)

addArchRGenome("hg38")
# Use for mouse samples
# addArchRGenome("mm10")

addArchRThreads(threads = 4)
bgzip("data/spatialatacrna/GSM6206884_HumanBrain_50um_fragments.tsv", 
      dest="data/spatialatacrna/GSM6206884_HumanBrain_50um_fragments.tsv.bgz", 
      overwrite=TRUE)
# create ArchR object
input_ATAC <- "data/spatialatacrna/GSM6206884_HumanBrain_50um_fragments.tsv.bgz"

ArrowFiles <- createArrowFiles(
  inputFiles = input_ATAC,
  sampleNames = sampleId,
  minTSS = 0,
  minFrags = 0,
  maxFrags = 1e+07,
  addTileMat = TRUE,
  addGeneScoreMat = TRUE,
  offsetPlus = 0,
  offsetMinus = 0,
  force = TRUE,
  TileMatParams = list(tileSize = 5000)
)
proj <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = sampleId,
  copyArrows = FALSE
)
# read the spatial barcode
barcode <- read.csv("data/spatialatacrna/HumanBrain_50um_spatial_ATAC/tissue_positions_list.csv",header = T,row.names = 1)
barcode <- barcode[-1,]
rownames(barcode) <- paste0(sampleId,"#",rownames(barcode),'-1')
archrindex <- c("array_col","array_row")
myindex <- c("X0","X0.1")
for (i in 1:2) {
    proj <- addCellColData(ArchRProj = proj, data=barcode[,myindex[i]], cells=rownames(barcode), name=archrindex[i], force=T)
}
proj <- subsetArchRProject(
    ArchRProj = proj,
  cells = rownames(barcode),
    force = TRUE
    )
# using spatial position to smooth
# for ATAC
proj <- IterativeLSI(
  ArchRProj = proj,
  useMatrix = "TileMatrix", 
  name = "LSI_ATAC", 
  iterations = 2, 
  clusterParams = list(
    resolution = c(2),
    sampleCells = NULL, 
    n.start = 10
  ), 
  varFeatures = 25000, 
  dimsToUse = 1:30,
  force = TRUE,
  saveIterations = FALSE,
  verbose = T, 
  logFile = "log.archr"
)
# add UMAP for RNA, ATAC and Combined
for (i in c("ATAC")) {
proj <- addUMAP(
  ArchRProj = proj, 
  reducedDims = paste0("LSI_", i),
  name = paste0("UMAP_",i),
  nNeighbors = 30, 
  minDist = 0.5, 
  metric = "cosine",
  force = TRUE)
}

# perform clustering for RNA, ATAC and Combined
for (i in c(0.3, 0.5, 0.8)){
  proj <-  IterativeLSI_Clustering(
    input = proj,
    reducedDims = "LSI_ATAC", 
    method = "Seurat",
    name = paste0("Clusters_ATAC_",i),
    resolution = i,
    prefix = "",
    force = TRUE,
    knnAssign = 30,
    nOutlier = 20,
    filterBias = T,
    verbose = F)
}
proj <- addGroupCoverages(
    proj,
    groupBy = "Clusters_ATAC_0.5"
    )
# Make sure to add your own path to Macs2
proj <- addReproduciblePeakSet(
    proj,
    groupBy = "Clusters_ATAC_0.5",
    pathToMacs2 = "/your/path/here"
    )
proj <- addPeakMatrix(proj)
proj <- addMotifAnnotations(ArchRProj = proj, motifSet = "cisbp", name = "Motif")
matches <- getMatches(proj)
peaks <- getMatrixFromProject(proj, "PeakMatrix")
saveArchRProject(ArchRProj = proj, outputDirectory = paste0("data/spatialatacrna/Save-",sampleId,".archr"), load = FALSE)
write.table(assay(matches), file = paste0("data/spatialatacrna/",sampleId,'_motifs.tsv'), quote=FALSE)
write.table(assay(peaks), file = paste0("data/spatialatacrna/",sampleId,'_peaks.tsv'), quote=FALSE)
write.csv(rowRanges(peaks), file = paste0("data/spatialatacrna/",sampleId,'_peak_ranges.csv'))